# Getting Started
**Make sure to clear the cell output before check into GitHub**
## Setup environment
If you don't have a .venv (Python 3.xx) Python kernel environment in the top right items of this window, then the first thing to do is setup the Python environment kernel:
- Go to View->Command Palette->Python: Create Environment... and run this command
- Select this Python kernel in top right of this window as your running environment

## Get Dataset
To training you need the dataset. The dataset for tennis ball can be found on Roboflow.
On Roboflow you can search for tennis dataset here: https://universe.roboflow.com/search?q=tennis+ball+model+object+detection+model%3Ayolov8

There are pretrained model with the dataset, so in some case if there is already a pretrained mdoel that is accurate enough for your use case, then you don't need to retrain the mdoel.]

The dataset that is used in this project is: https://universe.roboflow.com/viren-dhanwani/tennis-ball-detection/dataset/6

We will use YOLOv8 model, which is not the latest YOLO model, however it is small and efficient and with Ultralytics YOLO (v8), you can retain pretrained classes and add new specific classes by using a technique called partial fine-tuning or custom head training. The idea is to start from a pretrained model and train it to recognize both existing classes and your new ones — without forgetting the old ones (i.e., no catastrophic forgetting). Retain existing YOLO classes (like person, car, dog) from pretrained weights and add new custom class tennis_ball.

- Create a Roboflow account and log in
- Select "Download dataset" option
- Select "Show download code" option and accept the terms of agreement, you might need to go back to previous step if this is the first time you accept the agreement
- Copy the Jupyter code snipplet to below
- Replace the api_key with your API key from Roboflow under "Settings->API Keys"

Notes: When you run the first time, it might complained that you need a ipykernal environment as an error. Resolve the error by click on the Create Python Kernel environment. You can also go to View->Command Palette->Python: Create Environment...

In [11]:
%pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="wNKC4X8SBzsq9OEpxrLC")
# Downloading a YOLOv8 dataset for tennis ball detection
project = rf.workspace("viren-dhanwani").project("tennis-ball-detection")
version = project.version(6)
dataset = version.download("yolov8")

# Downloading a COCO dataset limited to person class
project2 = rf.workspace("shreks-swamp").project("coco-dataset-limited--person-only")
version2 = project2.version(1)
dataset2 = version2.download("yolov8")

Note: you may need to restart the kernel to use updated packages.
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to tennis-ball-detection-6 in yolov8:: 100%|██████████| 1168/1168 [00:00<00:00, 9651.06it/s]

loading Roboflow workspace...


loading Roboflow project...



Extracting Dataset Version Zip to COCO-Dataset-Limited-(Person-Only)-1 in yolov8:: 100%|██████████| 10498/10498 [00:00<00:00, 11363.96it/s]


## Training
We will training using YOLO model for object detection. YOLO is used since it is for object detection and tracking from frame to frame. To start training we need to setup the dataset directory in the right structure as expected by the training code.

The dataset has 3 directories:
- train: this is the images used to train the model
- test: this is the images used to test the model after training is done
- valid: this is used during the training process to tune model hyperparameters and prevent overfitting

Prepare Dataset with Extended Class List

Your data.yaml should list all the classes:

The original YOLOv8 classes (e.g., COCO has 80 classes) plus your custom new classes.
Example data.yaml:
```
    path: /path/to/dataset
    train: images/train
    val: images/val

    names:
    0: person
    1: car
    2: dog
    ...
    80: tennis ball
```
Current classes from YOLOv8 trained with COCO dataset are:
```
Class names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microwave', 69: 'oven', 70: 'toaster', 71: 'sink', 72: 'refrigerator', 73: 'book', 74: 'clock', 75: 'vase', 76: 'scissors', 77: 'teddy bear', 78: 'hair drier', 79: 'toothbrush'}
```
The label files also will need to be updated with the tennis_ball class (80) to match the data.yaml.

Now, we can train using Ultralytics API, which should be straight forward

In [ ]:
%pip install pyyaml

import os
def replace_first_column_in_files(directory_path, new_value):
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        
        # Only process regular files
        if os.path.isfile(file_path):
            with open(file_path, 'r') as file:
                lines = file.readlines()
            
            modified_lines = []
            for line in lines:
                parts = line.strip().split()
                if parts:
                    parts[0] = new_value
                    modified_lines.append(' '.join(parts) + '\n')
                else:
                    modified_lines.append('\n')  # Keep empty lines

            # Write back modified content
            with open(file_path, 'w') as file:
                file.writelines(modified_lines)

import yaml

# Class dictionary you want to add
class_names = {
    0: "person",
    1: "bicycle",
    2: "car",
    3: "motorcycle",
    4: "airplane",
    5: "bus",
    6: "train",
    7: "truck",
    8: "boat",
    9: "traffic light",
    10: "fire hydrant",
    11: "stop sign",
    12: "parking meter",
    13: "bench",
    14: "bird",
    15: "cat",
    16: "dog",
    17: "horse",
    18: "sheep",
    19: "cow",
    20: "elephant",
    21: "bear",
    22: "zebra",
    23: "giraffe",
    24: "backpack",
    25: "umbrella",
    26: "handbag",
    27: "tie",
    28: "suitcase",
    29: "frisbee",
    30: "skis",
    31: "snowboard",
    32: "sports ball",
    33: "kite",
    34: "baseball bat",
    35: "baseball glove",
    36: "skateboard",
    37: "surfboard",
    38: "tennis racket",
    39: "bottle",
    40: "wine glass",
    41: "cup",
    42: "fork",
    43: "knife",
    44: "spoon",
    45: "bowl",
    46: "banana",
    47: "apple",
    48: "sandwich",
    49: "orange",
    50: "broccoli",
    51: "carrot",
    52: "hot dog",
    53: "pizza",
    54: "donut",
    55: "cake",
    56: "chair",
    57: "couch",
    58: "potted plant",
    59: "bed",
    60: "dining table",
    61: "toilet",
    62: "tv",
    63: "laptop",
    64: "mouse",
    65: "remote",
    66: "keyboard",
    67: "cell phone",
    68: "microwave",
    69: "oven",
    70: "toaster",
    71: "sink",
    72: "refrigerator",
    73: "book",
    74: "clock",
    75: "vase",
    76: "scissors",
    77: "teddy bear",
    78: "hair drier",
    79: "toothbrush",
    80: "tennis ball"
}

# Path to your YAML file
yaml_path = dataset.location + "/data.yaml"

# Load the YAML file
with open(yaml_path, 'r') as file:
    data = yaml.safe_load(file)

# add new field 'names' to the YAML data
data['names'] = class_names
# Update the 'nc' value
data['nc'] = len(class_names)

# Save the updated YAML file
with open(yaml_path, 'w') as file:
    yaml.dump(data, file)

# Replace the first column in all label files with "80" which is the class id for tennis ball
replace_first_column_in_files(dataset.location + "/train/labels", "80")
replace_first_column_in_files(dataset.location + "/valid/labels", "80")
replace_first_column_in_files(dataset.location + "/test/labels", "80")

# combining the two datasets
# copy the COCO dataset to the tennis ball dataset
import shutil
def move_files(source_dir, destination_dir):
    os.makedirs(destination_dir, exist_ok=True)

    for filename in os.listdir(source_dir):
        src_path = os.path.join(source_dir, filename)
        dst_path = os.path.join(destination_dir, filename)
        if os.path.isfile(src_path):
            shutil.move(src_path, dst_path)

move_files(dataset2.location + "/test/images", dataset.location + "/test/images")
move_files(dataset2.location + "/test/labels", dataset.location + "/test/labels")
move_files(dataset2.location + "/train/images", dataset.location + "/train/images")
move_files(dataset2.location + "/train/labels", dataset.location + "/train/labels")
move_files(dataset2.location + "/valid/images", dataset.location + "/valid/images")
move_files(dataset2.location + "/valid/labels", dataset.location + "/valid/labels")

Note: you may need to restart the kernel to use updated packages.
Moved: 000000300782_jpg.rf.af651ca9574c2e66952974250ff6fb1b.jpg
Moved: 000000243950_jpg.rf.a303740d7db3dc9d2f9f42402cdc29bc.jpg
Moved: 000000080448_jpg.rf.ff6fd275fa433b4f168f489d0978f45d.jpg
Moved: 000000420960_jpg.rf.c22e777db57db7f499b55c9c1c445af6.jpg
Moved: 000000229058_jpg.rf.6e3a3b2fca74587ee49ae2dc3f414d83.jpg
Moved: 000000177957_jpg.rf.c05180e2b91813c658e8e3db5479bf8a.jpg
Moved: 000000359066_jpg.rf.ee695ef364b84d582721ace33d9e8db6.jpg
Moved: 000000009885_jpg.rf.5700dfd717fe1dd29133fbd261ef54d2.jpg
Moved: 000000178645_jpg.rf.b4c0ce07a728c616bfe98e64256e6470.jpg
Moved: 000000248468_jpg.rf.2c4126fad2325c3b38908ef97595ff4b.jpg
Moved: 000000310013_jpg.rf.0cc044cea47c2575dce433f223977c03.jpg
Moved: 000000319798_jpg.rf.bee6d4a62849322d076628e489172433.jpg
Moved: 000000198493_jpg.rf.ed2a07b917f3afd14fd3e15931c4d8c0.jpg
Moved: 000000401194_jpg.rf.e8d16adc308f012074e520eaeafe60ca.jpg
Moved: 000000039509_jpg.rf.84eaba87400

## Start training process
To train using Ultralytics API, which should be straight forward with the number of epochs and image size. This will probably not run on the laptop without Nvidia GPU.
Doc on Yolov8 https://docs.ultralytics.com/models/yolov8/#performance-metrics

There are many Yolo v8 models. The 2 opposite models are yolov8xu.pt and yolov8nu.pt.
- yolov8x.pt where 'x' is extra large parameters which is used for high accuracy
- yolov8n.pt where 'n' is nano parametrs which is small and fast but lack accuracy. We will use this model since our use case is for the device

In [ ]:
%pip install ultralytics
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO("yolov8n.pt")  # Load the YOLOv8 nano model
print("data set location:", dataset.location)
# Train the model on the dataset
model.train(data=dataset.location + "/data.yaml", epochs=100, imgsz=640)


## Google Collab
This will probably not run on the laptop, so you should transfer the code to Google Collab to run on a GPU:
- Create and login account on Google Collab https://colab.research.google.com/
- Upload this notebook by drag-n-drop the file into the File->Open notebook dialog
- Change your runtime to T4 GPU under Runtime->Change runtime type
- Run all the start the training and output should be like:
```
    Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      12/100      14.5G      3.171      2.148     0.9224         30        640: 100%|██████████| 27/27 [00:35<00:00,  1.32s/it]
                  Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.97it/s]                   all        100        101      0.548      0.297       0.23     0.0472
```
- After the run, the models need to be copied to your Google drive. The below code is only needed if you are running this in Google Collab
- Make sure you create a folder "my_models" under "My Drive" of your Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp -r /content/runs/detect/train* /content/drive/MyDrive/my_models/